In [21]:
from dotenv import load_dotenv
import os
from pathlib import Path
from datetime import datetime, timedelta

import torch
from googleapiclient.discovery import build
import yt_dlp
import whisper
import re
from tqdm import tqdm

import google.generativeai as genai
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

import pandas as pd
from pathlib import Path
import os
import shutil

In [35]:
folder_path = './preprocessed_data/llm/predict_text'
path_obj = Path(folder_path)

if not path_obj.is_dir():
    print(f"오류: '{folder_path}'는 유효한 폴더 경로가 아닙니다.")

print(f"\n'{folder_path}' 내의 모든 파일 목록 (pathlib 사용):")

dfs = []
for file_path in path_obj.rglob('*.csv'): 
    if file_path.is_file(): # 파일만 선택
        df = pd.read_csv(file_path, encoding='utf-8')
        dfs.append(df)
        
df_t = pd.concat(dfs)
df_t = df_t.sort_values(by=['code', 'year', 'month', 'week'], ascending=True, ignore_index=True)
df_t.to_csv('./preprocessed_data/llm/predict_total/predict_text.csv', encoding='utf-8', index=False)


'./preprocessed_data/llm/predict_text' 내의 모든 파일 목록 (pathlib 사용):


In [36]:
folder_path = './preprocessed_data/llm/predict_video'
path_obj = Path(folder_path)

if not path_obj.is_dir():
    print(f"오류: '{folder_path}'는 유효한 폴더 경로가 아닙니다.")

print(f"\n'{folder_path}' 내의 모든 파일 목록 (pathlib 사용):")

dfs = []
for file_path in path_obj.rglob('*.csv'): 
    if file_path.is_file(): # 파일만 선택
        df = pd.read_csv(file_path, encoding='utf-8')
        dfs.append(df)
        
df_v = pd.concat(dfs)
df_t = df_t.sort_values(by=['code', 'year', 'month', 'week'], ascending=True, ignore_index=True)
df_v.to_csv('./preprocessed_data/llm/predict_total/predict_video.csv', encoding='utf-8', index=False)


'./preprocessed_data/llm/predict_video' 내의 모든 파일 목록 (pathlib 사용):


## 영상 자료가 없는 것 -> 기사로 대체 

In [37]:
df_v = pd.read_csv('./preprocessed_data/llm/predict_total/predict_video.csv', encoding='utf-8')
df_t = pd.read_csv('./preprocessed_data/llm/predict_total/predict_text.csv', encoding='utf-8')

nan_list = pd.isna(df_v['prediction'])
not_nan_list = ~pd.isna(df_t['prediction'])
df_v[nan_list & not_nan_list] = df_t[nan_list & not_nan_list]
df_v.to_csv('./preprocessed_data/llm/predict_total/predict_mix.csv', encoding='utf-8', index=False)

## 영상 자료 없는 것 + 중립 -> 기사로 대체

In [38]:
df_v = pd.read_csv('./preprocessed_data/llm/predict_total/predict_video.csv', encoding='utf-8')
df_t = pd.read_csv('./preprocessed_data/llm/predict_total/predict_text.csv', encoding='utf-8')

nan_list = pd.isna(df_v['prediction']) | (df_v['prediction'] == '불가능') | (df_v['prediction'] == '중립')
not_nan_list = ~(pd.isna(df_t['prediction']) | (df_t['prediction'] == '불가능') | (df_t['prediction'] == '중립'))
df_v[nan_list & not_nan_list] = df_t[nan_list & not_nan_list]
df_v.to_csv('./preprocessed_data/llm/predict_total/predict_mix2.csv', encoding='utf-8', index=False)

## 자료 수집 결측치 확인

In [39]:
print('결측치 비율')
df_v = pd.read_csv('./preprocessed_data/llm/predict_total/predict_video.csv', encoding='utf-8')
df_t = pd.read_csv('./preprocessed_data/llm/predict_total/predict_text.csv', encoding='utf-8')
df_m = pd.read_csv('./preprocessed_data/llm/predict_total/predict_mix.csv', encoding='utf-8')
df_m2 = pd.read_csv('./preprocessed_data/llm/predict_total/predict_mix2.csv', encoding='utf-8')

nan_list = pd.isna(df_v['prediction'])
nan_num = sum(nan_list)
total_num = len(df_v)
print(f'video\t\t결측: {nan_num} / {total_num}')

nan_list = pd.isna(df_t['prediction'])
nan_num = sum(nan_list)
total_num = len(df_t)
print(f'text\t\t결측: {nan_num} / {total_num}')

nan_list = pd.isna(df_m['prediction'])
nan_num = sum(nan_list)
total_num = len(df_m)
print(f'mix1\t\t결측: {nan_num} / {total_num}')

nan_list = pd.isna(df_m2['prediction'])
nan_num = sum(nan_list)
total_num = len(df_m2)
print(f'mix2\t\t결측: {nan_num} / {total_num}')

############################################################

print('\n중립 비율')
df_v = pd.read_csv('./preprocessed_data/llm/predict_total/predict_video.csv', encoding='utf-8')
df_t = pd.read_csv('./preprocessed_data/llm/predict_total/predict_text.csv', encoding='utf-8')
df_m = pd.read_csv('./preprocessed_data/llm/predict_total/predict_mix.csv', encoding='utf-8')
df_m2 = pd.read_csv('./preprocessed_data/llm/predict_total/predict_mix2.csv', encoding='utf-8')

nan_list = (df_v['prediction'] == '중립') | (df_v['prediction'] == '불가능')
nan_num = sum(nan_list)
total_num = len(df_v)
print(f'video\t\t중립: {nan_num} / {total_num}')

nan_list = (df_t['prediction'] == '중립') | (df_t['prediction'] == '불가능')
nan_num = sum(nan_list)
total_num = len(df_t)
print(f'text\t\t중립: {nan_num} / {total_num}')

nan_list = (df_m['prediction'] == '중립') | (df_m['prediction'] == '불가능')
nan_num = sum(nan_list)
total_num = len(df_m)
print(f'mix1\t\t중립: {nan_num} / {total_num}')

nan_list = (df_m2['prediction'] == '중립') | (df_m2['prediction'] == '불가능')
nan_num = sum(nan_list)
total_num = len(df_m2)
print(f'mix2\t\t중립: {nan_num} / {total_num}')

############################################################

print('\n결측치+중립 비율')
df_v = pd.read_csv('./preprocessed_data/llm/predict_total/predict_video.csv', encoding='utf-8')
df_t = pd.read_csv('./preprocessed_data/llm/predict_total/predict_text.csv', encoding='utf-8')
df_m = pd.read_csv('./preprocessed_data/llm/predict_total/predict_mix.csv', encoding='utf-8')
df_m2 = pd.read_csv('./preprocessed_data/llm/predict_total/predict_mix2.csv', encoding='utf-8')

nan_list = pd.isna(df_v['prediction']) | (df_v['prediction'] == '중립') | (df_v['prediction'] == '불가능')
nan_num = sum(nan_list)
total_num = len(df_v)
print(f'video\t\t결측: {nan_num} / {total_num}')

nan_list = pd.isna(df_t['prediction']) | (df_t['prediction'] == '중립') | (df_t['prediction'] == '불가능')
nan_num = sum(nan_list)
total_num = len(df_t)
print(f'text\t\t결측: {nan_num} / {total_num}')

nan_list = pd.isna(df_m['prediction']) | (df_m['prediction'] == '중립') | (df_m['prediction'] == '불가능')
nan_num = sum(nan_list)
total_num = len(df_m)
print(f'mix1\t\t결측: {nan_num} / {total_num}')

nan_list = pd.isna(df_m2['prediction']) | (df_m2['prediction'] == '중립') | (df_m2['prediction'] == '불가능')
nan_num = sum(nan_list)
total_num = len(df_m2)
print(f'mix2\t\t결측: {nan_num} / {total_num}')

결측치 비율
video		결측: 2046 / 5712
text		결측: 1710 / 5712
mix1		결측: 600 / 5712
mix2		결측: 1269 / 5712

중립 비율
video		중립: 2380 / 5712
text		중립: 1877 / 5712
mix1		중립: 3049 / 5712
mix2		중립: 1515 / 5712

결측치+중립 비율
video		결측: 4426 / 5712
text		결측: 3587 / 5712
mix1		결측: 3649 / 5712
mix2		결측: 2784 / 5712


In [ ]:
sectors=['산업재', '정보기술']
df = pd.read_csv('preprocessed_data/llm/predict_total/predict_mix2.csv', encoding='utf-8')

# 영상 업로드 날짜 열 생성
df_video = pd.read_csv("data_kr/video/동영상 수집 통합본 최신.csv")
df=df.sort_values(by=['code', 'year', 'quarter']).reset_index(drop=True)
df_video=df_video.sort_values(by=['code', 'year', 'quarter']).reset_index(drop=True)
df["upload_date"] = df_video["upload_date"]

### LLM 예측에 실제 등락 라벨 추가 ###
df["code"] = df["code"].astype(str).str.zfill(6)

price_upload = [] # 업로드 당일 종가
price_closure = [] # 공시 당일 종가
for row in tqdm(df.itertuples(), total=len(df)):
    df_price = pd.read_csv(f"data_kr/price/{row.code}.csv")
    ### 업로드 날짜 직전 종가
    price_upload.append(df_price.loc[df_price["날짜"] < row.upload_date, "종가"].iloc[-1] if not pd.isna(row.upload_date) else None)
    ### 공시일 날짜 직후 종가
    price_closure.append(df_price.loc[df_price["날짜"] >= row.disclosure_date, "종가"].iloc[0] if not pd.isna(row.disclosure_date) else None)

df["price_upload"] = price_upload
df["price_closure"] = price_closure
label = df["price_upload"] > df["price_closure"] # (업로드 당일 종가)보다 (공시 당일 종가)가 올랐는가?
df["label"] = ["up" if l == True else "down" if l == False else None for l in label]
df.loc[pd.isna(df['upload_date']), 'label'] = pd.NA

df = df[["code", "name", "sector", "year", "quarter", "upload_date", "disclosure_date", "price_upload", "price_closure", "prediction", "label"]]
df.to_csv("preprocessed_data/llm/predict/업로드_공시_가격비교.csv", index=False, encoding="utf-8")

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# 데이터
label = df['label'].dropna()
prediction = df['prediction'].dropna()

# 혼동 행렬
cm = confusion_matrix(label, prediction, labels=["up", "down"])

# 시각화
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["up", "down"], yticklabels=["up", "down"])
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.title('Confusion Matrix')
plt.savefig("preprocessed_data/llm/predict/confusion_matrix.png")